<a href="https://colab.research.google.com/github/kgndirangu/project-4/blob/main/healthcare_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 4s (61.1 kB/s)
Reading package lists... Done


In [ ]:
!pip install pyarrow
!pip install memory_profiler


In [ ]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import memory_profiler

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# Read in data from S3 Buckets

url = "https://project4-ny-health-data-1.s3.amazonaws.com/hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show Spark DataFrame
df.show()

+-----+-------------------+---------------+----------------------------+-----------+--------------------+-----------+-------------------+------+--------------------+-----------------+--------------+-----------------+--------------------+--------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+--------------------+-------------------+-------------------+---------------------------------+---------------------------------+-----------------------------+------------+-----------------------+------------------------------+-------------+-----------+
|index|Health Service Area|Hospital County|Operating Certificate Number|Facility ID|       Facility Name|  Age Group|Zip Code - 3 digits|Gender|                Race|        Ethnicity|Length of Stay|Type of Admission|

In [ ]:
df.dtypes

[('index', 'int'),
 ('Health Service Area', 'string'),
 ('Hospital County', 'string'),
 ('Operating Certificate Number', 'double'),
 ('Facility ID', 'double'),
 ('Facility Name', 'string'),
 ('Age Group', 'string'),
 ('Zip Code - 3 digits', 'string'),
 ('Gender', 'string'),
 ('Race', 'string'),
 ('Ethnicity', 'string'),
 ('Length of Stay', 'string'),
 ('Type of Admission', 'string'),
 ('Patient Disposition', 'string'),
 ('Discharge Year', 'int'),
 ('CCS Diagnosis Code', 'double'),
 ('CCS Diagnosis Description', 'string'),
 ('CCS Procedure Code', 'double'),
 ('CCS Procedure Description', 'string'),
 ('APR DRG Code', 'int'),
 ('APR DRG Description', 'string'),
 ('APR MDC Code', 'int'),
 ('APR MDC Description', 'string'),
 ('APR Severity of Illness Code', 'int'),
 ('APR Severity of Illness Description', 'string'),
 ('APR Risk of Mortality', 'string'),
 ('APR Medical Surgical Description', 'string'),
 ('Source of Payment 1', 'string'),
 ('Source of Payment 2', 'string'),
 ('Source of Payme

index 2622133

Health Service Area 9

Hospital County 58

Operating Certificate Number 190

Facility ID 226

Facility Name 225

Age Group 5

Zip Code - 3 digits 76

Gender 3

Race 4

Ethnicity 3

Length of Stay 120

Type of Admission 6

Patient Disposition 20

Discharge Year 1

CCS Diagnosis Code 263

CCS Diagnosis Description 263

CCS Procedure Code 233

CCS Procedure Description 233

APR DRG Code 316

APR DRG Description 316

APR MDC Code 26

APR MDC Description 26

APR Severity of Illness Code 5

APR Severity of Illness Description 5

APR Risk of Mortality 5

APR Medical Surgical Description 3

Source of Payment 1 10

Source of Payment 2 11

Source of Payment 3 11

Attending Provider License Number 30087

Operating Provider License Number 36249

Other Provider License Number 12650

Birth Weight 91

Abortion Edit Indicator 2

Emergency Department Indicator 2

Total Charges 1751253

Total Costs 1429694






In [ ]:
#Count of categories in each column
#6 minute run time
#for c in df.columns:
#  print(c, df.select(c).distinct().count())

index 2622133
Health Service Area 9
Hospital County 58
Operating Certificate Number 190
Facility ID 226
Facility Name 225
Age Group 5
Zip Code - 3 digits 76
Gender 3
Race 4
Ethnicity 3
Length of Stay 120
Type of Admission 6
Patient Disposition 20
Discharge Year 1
CCS Diagnosis Code 263
CCS Diagnosis Description 263
CCS Procedure Code 233
CCS Procedure Description 233
APR DRG Code 316
APR DRG Description 316
APR MDC Code 26
APR MDC Description 26
APR Severity of Illness Code 5
APR Severity of Illness Description 5
APR Risk of Mortality 5
APR Medical Surgical Description 3
Source of Payment 1 10
Source of Payment 2 11
Source of Payment 3 11
Attending Provider License Number 30087
Operating Provider License Number 36249
Other Provider License Number 12650
Birth Weight 91
Abortion Edit Indicator 2
Emergency Department Indicator 2
Total Charges 1751253
Total Costs 1429694


In [ ]:
#Count columns
len(df.columns)


38

In [ ]:
df.select(["Total Costs", "Total Charges"]).describe().show()

+-------+------------------+------------------+
|summary|       Total Costs|     Total Charges|
+-------+------------------+------------------+
|  count|           2622133|           2622133|
|   mean|11865.230540174922|29459.201117857512|
| stddev|27270.287332340587|56400.209381384615|
|    min|               0.0|              0.01|
|    max|     1.470885259E7|     1.206004382E7|
+-------+------------------+------------------+



In [ ]:
#Count rows
df.count()

2622133

In [ ]:
#############Describe data set
#https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.describe.html
df.summary

<bound method DataFrame.summary of DataFrame[index: int, Health Service Area: string, Hospital County: string, Operating Certificate Number: double, Facility ID: double, Facility Name: string, Age Group: string, Zip Code - 3 digits: string, Gender: string, Race: string, Ethnicity: string, Length of Stay: string, Type of Admission: string, Patient Disposition: string, Discharge Year: int, CCS Diagnosis Code: double, CCS Diagnosis Description: string, CCS Procedure Code: double, CCS Procedure Description: string, APR DRG Code: int, APR DRG Description: string, APR MDC Code: int, APR MDC Description: string, APR Severity of Illness Code: int, APR Severity of Illness Description: string, APR Risk of Mortality: string, APR Medical Surgical Description: string, Source of Payment 1: string, Source of Payment 2: string, Source of Payment 3: string, Attending Provider License Number: double, Operating Provider License Number: double, Other Provider License Number: double, Birth Weight: int, Abortion Edit Indicator: string, Emergency Department Indicator: string, Total Charges: double, Total Costs: double]>

In [ ]:
#Count null values

for column in df.columns:
    null_count = df.filter(col(column).isNull()).count()
    print(f"Null count in {column}: {null_count}")

Null count in index: 0
Null count in Health Service Area: 4887
Null count in Hospital County: 4887
Null count in Operating Certificate Number: 4887
Null count in Facility ID: 4887
Null count in Facility Name: 0
Null count in Age Group: 0
Null count in Zip Code - 3 digits: 3201
Null count in Gender: 0
Null count in Race: 0
Null count in Ethnicity: 0
Null count in Length of Stay: 0
Null count in Type of Admission: 0
Null count in Patient Disposition: 103
Null count in Discharge Year: 0
Null count in CCS Diagnosis Code: 2273
Null count in CCS Diagnosis Description: 2273
Null count in CCS Procedure Code: 2273
Null count in CCS Procedure Description: 2273
Null count in APR DRG Code: 0
Null count in APR DRG Description: 0
Null count in APR MDC Code: 0
Null count in APR MDC Description: 0
Null count in APR Severity of Illness Code: 0
Null count in APR Severity of Illness Description: 241
Null count in APR Risk of Mortality: 241
Null count in APR Medical Surgical Description: 0
Null count in S

In [ ]:
df_reduced = df.drop("index", "Operating Certificate Number", "Discharge Year", "Source of Payment 2", "Source of Payment 3", "Attending Provider License Number", "Operating Provider License Number",
                     "Other Provider License Number", "Birth Weight", "Abortion Edit Indicator")

In [ ]:
df_reduced.show()

+-------------------+---------------+-----------+--------------------+-----------+-------------------+------+--------------------+-----------------+--------------+-----------------+--------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+--------------------+------------------------------+-------------+-----------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|  Age Group|Zip Code - 3 digits|Gender|                Race|        Ethnicity|Length of Stay|Type of Admission| Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|CCS Procedure Description|APR DRG Code| APR DRG Description|APR MDC Code| APR MDC Description|APR Severity of Illness Code|APR Severity of Illness Description|APR Risk of Mort

In [ ]:
# Drop rows that have null values in any column
df_clean = df_reduced.na.drop()

# Show the result to verify that rows with null values have been removed
df_clean.show()

+-------------------+---------------+-----------+--------------------+---------+-------------------+------+--------------------+-----------------+--------------+-----------------+--------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+--------------------+------------------------------+-------------+-----------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|Age Group|Zip Code - 3 digits|Gender|                Race|        Ethnicity|Length of Stay|Type of Admission| Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|CCS Procedure Description|APR DRG Code| APR DRG Description|APR MDC Code| APR MDC Description|APR Severity of Illness Code|APR Severity of Illness Description|APR Risk of Mortalit

In [ ]:
#Count rows after dropping null values
df_clean.count()

2611433

In [ ]:
#Count columns
len(df_clean.columns)

28

In [ ]:
df_clean.withColumn("Length of Stay",col("Length of Stay").cast("int"))
df_clean.withColumn("Zip Code - 3 digits",col("Zip Code - 3 digits").cast("int"))

DataFrame[Health Service Area: string, Hospital County: string, Facility ID: double, Facility Name: string, Age Group: string, Zip Code - 3 digits: int, Gender: string, Race: string, Ethnicity: string, Length of Stay: string, Type of Admission: string, Patient Disposition: string, CCS Diagnosis Code: double, CCS Diagnosis Description: string, CCS Procedure Code: double, CCS Procedure Description: string, APR DRG Code: int, APR DRG Description: string, APR MDC Code: int, APR MDC Description: string, APR Severity of Illness Code: int, APR Severity of Illness Description: string, APR Risk of Mortality: string, APR Medical Surgical Description: string, Source of Payment 1: string, Emergency Department Indicator: string, Total Charges: double, Total Costs: double]

Exploring categorical variables

In [ ]:
#9 categories of health service area
df.groupBy("Health Service Area").count().show(n=100)

+-------------------+-------+
|Health Service Area|  count|
+-------------------+-------+
|         Central NY| 170912|
|         Western NY| 185080|
|     Capital/Adiron| 179929|
|      Southern Tier|  31300|
|      Hudson Valley| 273037|
|        Long Island| 385262|
|       Finger Lakes| 158605|
|      New York City|1233121|
|               NULL|   4887|
+-------------------+-------+



In [ ]:
#58 categories of hospital counties
df.groupBy("Hospital County").count().orderBy('count', ascending=False).show(n=20)

+---------------+------+
|Hospital County| count|
+---------------+------+
|      Manhattan|445288|
|          Kings|298247|
|         Queens|221325|
|         Nassau|215831|
|          Bronx|204328|
|        Suffolk|169431|
|    Westchester|131946|
|           Erie|129617|
|         Monroe|109402|
|       Onondaga| 76289|
|       Richmond| 63933|
|         Albany| 62192|
|         Orange| 42955|
|         Oneida| 36454|
|       Dutchess| 35424|
|       Rockland| 34898|
|         Broome| 29078|
|    Schenectady| 22989|
|        Niagara| 21450|
|        Chemung| 17366|
+---------------+------+
only showing top 20 rows



In [ ]:
#76 zip codes of the patient
df.groupBy("Zip Code - 3 digits").count().orderBy('count', ascending=False).show(n=20)

+-------------------+------+
|Zip Code - 3 digits| count|
+-------------------+------+
|                112|364357|
|                104|243938|
|                117|202733|
|                100|201754|
|                113|139877|
|                115|102381|
|                114|100309|
|                OOS| 74242|
|                142| 72432|
|                105| 68536|
|                103| 68045|
|                109| 63469|
|                146| 58500|
|                125| 50583|
|                140| 49984|
|                119| 36746|
|                130| 35603|
|                107| 35553|
|                145| 32462|
|                144| 31461|
+-------------------+------+
only showing top 20 rows



In [ ]:
#120 categories of legnth of stay
df.groupBy("Length of Stay").count().show(n=100)

+--------------+------+
|Length of Stay| count|
+--------------+------+
|             7|104813|
|            51|   602|
|            15| 19087|
|            54|   530|
|            11| 35890|
|           101|    67|
|            29|  3428|
|            69|   255|
|            42|  1206|
|           112|    54|
|            87|   115|
|            73|   213|
|            64|   337|
|             3|449094|
|            30|  2872|
|           113|    71|
|            34|  2057|
|            59|   722|
|             8| 76128|
|            22|  7323|
|            28|  6552|
|            85|   159|
|            16| 14965|
|            35|  2075|
|            52|   559|
|            71|   256|
|            98|   111|
|            47|   786|
|            99|    75|
|           107|    55|
|           110|    63|
|            96|    89|
|            43|  1087|
|             5|178402|
|            31|  2431|
|           100|    88|
|            18| 11036|
|            70|   281|
|            27|

In [ ]:
df.groupBy("Age Group").count().show(n=100)

+-----------+------+
|  Age Group| count|
+-----------+------+
|    0 to 17|387353|
|   18 to 29|281978|
|   50 to 69|680166|
|70 or Older|725253|
|   30 to 49|547383|
+-----------+------+



In [ ]:
df.groupBy("Race").count().show(n=100)

+--------------------+-------+
|                Race|  count|
+--------------------+-------+
|          Other Race| 493854|
|             Unknown|  28340|
|               White|1601378|
|Black/African Ame...| 498561|
+--------------------+-------+



In [ ]:
df.groupBy("Ethnicity").count().show(n=100)

+-----------------+-------+
|        Ethnicity|  count|
+-----------------+-------+
|Not Span/Hispanic|2116780|
|          Unknown| 153744|
| Spanish/Hispanic| 351609|
+-----------------+-------+



In [ ]:
#20 categories, consider reducing since a few have <10,000
df.groupBy("Patient Disposition").count().show(n=100)

+--------------------+-------+
| Patient Disposition|  count|
+--------------------+-------+
|Hospice - Medical...|   8635|
|Court/Law Enforce...|   3324|
|Facility w/ Custo...|   7291|
|Another Type Not ...|   8540|
|Skilled Nursing Home| 246148|
|                NULL|    103|
|Medicare Cert Lon...|   3475|
|   Home or Self Care|1786141|
|Cancer Center or ...|   5035|
|Inpatient Rehabil...|  44489|
|Home w/ Home Heal...| 321896|
|             Expired|  56160|
|Psychiatric Hospi...|  15320|
|Medicaid Cert Nur...|     46|
|Hosp Basd Medicar...|   2056|
|Left Against Medi...|  56948|
| Short-term Hospital|  47215|
|Federal Health Ca...|    718|
|Critical Access H...|     44|
|      Hospice - Home|   8549|
+--------------------+-------+



In [ ]:
#316 categories
df.groupBy("APR DRG Description").count().orderBy('count', ascending=False).show(n=20)

+--------------------+------+
| APR DRG Description| count|
+--------------------+------+
|NEONATE BIRTHWT >...|210893|
|    VAGINAL DELIVERY|152443|
|   CESAREAN DELIVERY| 81812|
|       HEART FAILURE| 57440|
|SEPTICEMIA & DISS...| 53025|
|     OTHER PNEUMONIA| 51231|
|CHRONIC OBSTRUCTI...| 48109|
|CELLULITIS & OTHE...| 42264|
|CARDIAC ARRHYTHMI...| 40354|
|          CHEST PAIN| 39776|
|      REHABILITATION| 38170|
|PERCUTANEOUS CARD...| 37696|
|KIDNEY & URINARY ...| 35035|
|       SCHIZOPHRENIA| 32885|
|KNEE JOINT REPLAC...| 32212|
|  SYNCOPE & COLLAPSE| 31922|
|ANGINA PECTORIS &...| 30331|
|   BIPOLAR DISORDERS| 30287|
|MAJOR DEPRESSIVE ...| 29706|
|NON-BACTERIAL GAS...| 28559|
+--------------------+------+
only showing top 20 rows



In [ ]:
#26 categories
df.groupBy('APR MDC Description').count().orderBy('count', ascending=False).show(n=20)

+--------------------+------+
| APR MDC Description| count|
+--------------------+------+
|Diseases and Diso...|396907|
|Pregnancy, Childb...|273123|
|Newborns and Othe...|246173|
|Diseases and Diso...|233412|
|Diseases and Diso...|221189|
|Diseases and Diso...|202225|
|Diseases and Diso...|149874|
|Mental Diseases a...|121808|
|Diseases and Diso...|110336|
|Alcohol/Drug Use ...| 89385|
|Infectious and Pa...| 87763|
|Endocrine, Nutrit...| 78191|
|Diseases and Diso...| 74665|
|Diseases and Diso...| 69334|
|Rehabilitation, A...| 54373|
|Diseases and Diso...| 43194|
|Diseases and Diso...| 39025|
|Ear, Nose, Mouth,...| 36839|
|Poisonings, Toxic...| 32205|
|Lymphatic, Hemato...| 25702|
+--------------------+------+
only showing top 20 rows



In [ ]:
#5 categories
df.groupBy('APR Severity of Illness Description').count().orderBy('count', ascending=False).show(n=20)

+-----------------------------------+------+
|APR Severity of Illness Description| count|
+-----------------------------------+------+
|                              Minor|974425|
|                           Moderate|962658|
|                              Major|530085|
|                            Extreme|154724|
|                               NULL|   241|
+-----------------------------------+------+



In [ ]:
#5 categories
df.groupBy('APR Risk of Mortality').count().orderBy('count', ascending=False).show(n=20)

+---------------------+-------+
|APR Risk of Mortality|  count|
+---------------------+-------+
|                Minor|1644151|
|             Moderate| 554542|
|                Major| 300829|
|              Extreme| 122370|
|                 NULL|    241|
+---------------------+-------+



In [ ]:
#3 categories
df.groupBy('APR Medical Surgical Description').count().orderBy('count', ascending=False).show(n=20)

+--------------------------------+-------+
|APR Medical Surgical Description|  count|
+--------------------------------+-------+
|                         Medical|2002344|
|                        Surgical| 619548|
|                  Not Applicable|    241|
+--------------------------------+-------+



In [ ]:
#263 categories
df.groupBy('CCS Diagnosis Description').count().orderBy('count', ascending=False).show(n=20)

+-------------------------+------+
|CCS Diagnosis Description| count|
+-------------------------+------+
|                 LIVEBORN|241123|
|           MOOD DISORDERS| 63068|
|                      CHF| 62241|
|               SEPTICEMIA| 60989|
|               CHEST PAIN| 60055|
|     CORONARY ATHEROSCLER| 59440|
|                PNEUMONIA| 57579|
|              DYSRHYTHMIA| 54446|
|           OSTEOARTHRITIS| 50369|
|     ALCOHOL-RELATD DI...| 48982|
|     SKIN/SUBCUT TISS ...| 44259|
|     SUBSTANCE-RLTD DI...| 43555|
|     OTH COMP BIRTH/PU...| 42187|
|     DEVICE/IMPLNT/GRF...| 41238|
|     SCHIZ/OTH PSYCH D...| 40444|
|         DIABETES W/COMPL| 39075|
|                   ASTHMA| 38856|
|     REHAB/PROSTH FIT/...| 38395|
|     OB PERINEAL/VULV ...| 37888|
|                     COPD| 37568|
+-------------------------+------+
only showing top 20 rows



In [ ]:
#233 categories
#Set threshold, FOR EXAMPLE <30,000, convert remaining to 'Other' to reduce categories to 20 or less
df.groupBy('CCS Procedure Description').count().orderBy('count', ascending=False).show(n=20)

+-------------------------+------+
|CCS Procedure Description| count|
+-------------------------+------+
|                  NO PROC|665758|
|     OTHER THERAPEUTIC...|159957|
|         CESAREAN SECTION| 81491|
|     ALCO/DRUG REHAB/D...| 75161|
|     PROPHYLACTIC VAC/...| 70832|
|     RESP INTUB/MECH V...| 65991|
|     OT PRCS TO ASSIST...| 65302|
|     REPAIR CUR OBS LA...| 56677|
|             CIRCUMCISION| 54895|
|     OPHTHALM-/OT-OLOG...| 51537|
|        BLOOD TRANSFUSION| 51187|
|     PSYCHO/PSYCHI EVA...| 47823|
|     PERC TRANSLUM COR...| 42069|
|      DX ULTRASOUND HEART| 38993|
|     COMP AXIAL TOMOGR...| 36812|
|     UP GASTRO ENDOSC/...| 36544|
|        ARTHROPLASTY KNEE| 32502|
|     DX CARDIAC CATHET...| 31603|
|       OTHER RESP THERAPY| 29740|
|     HIP REPLACEMENT,T...| 26192|
+-------------------------+------+
only showing top 20 rows



Encode categorical variables of interest

In [ ]:
df_manhattan.count()

443902

In [ ]:
df_manhattan.show()

+-------------------+---------------+-----------+--------------------+---------+-------------------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+-----------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|Age Group|Zip Code - 3 digits|Gender| Race|        Ethnicity|Length of Stay|Type of Admission|Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|CCS Procedure Description|APR DRG Code| APR DRG Description|APR MDC Code| APR MDC Description|APR Severity of Illness Code|APR Severity of Illness Description|APR Risk of Mortality|APR Medical Surgical Descriptio

In [ ]:
df_manhattan = df_clean.filter(col('Hospital County') == "Manhattan")

In [ ]:
#Encode categorical columns of interest Health Service Area, Age Group, Zip Code - 3 digits, Gender, Race, Emergency Department Indicator, Source of Payment 1-3,
# Type of Admission, Patient Disposition, APR Risk of Mortality, APR Medical Surgical Description

strIndxAge = StringIndexer(inputCol = "Age Group", outputCol="AgeIndex")
encAge = OneHotEncoder(inputCol="AgeIndex", outputCol="catVecAge")

strIndxSex = StringIndexer(inputCol = "Gender", outputCol="SexIndex")
encSex = OneHotEncoder(inputCol="SexIndex", outputCol="catVecSex")

strIndxRace = StringIndexer(inputCol = "Race", outputCol="RaceIndex")
encRace = OneHotEncoder(inputCol="RaceIndex", outputCol="catVecRace")

strIndxER = StringIndexer(inputCol = "Emergency Department Indicator", outputCol="ERIndex")
encER = OneHotEncoder(inputCol="ERIndex", outputCol="catVecER")

strIndxPay1 = StringIndexer(inputCol = "Source of Payment 1", outputCol="Pay1Index")
encPay1 = OneHotEncoder(inputCol="Pay1Index", outputCol="catVecPay1")

strIndxAdm = StringIndexer(inputCol = "Type of Admission", outputCol="AdmIndex")
encAdm = OneHotEncoder(inputCol="AdmIndex", outputCol="catVecAdm")

strIndxDisp = StringIndexer(inputCol = "Patient Disposition", outputCol="DispIndex")
encDisp = OneHotEncoder(inputCol="DispIndex", outputCol="catVecDisp")

strIndxMort = StringIndexer(inputCol = "APR Risk of Mortality", outputCol="MortIndex")
encMort = OneHotEncoder(inputCol="MortIndex", outputCol="catVecMort")

strIndxSurg = StringIndexer(inputCol = "APR Medical Surgical Description", outputCol="SurgIndex")
encSurg = OneHotEncoder(inputCol="SurgIndex", outputCol="catVecSurg")

pipeline = Pipeline(stages=[strIndxAge, encAge, strIndxSex, encSex, strIndxRace, encRace, strIndxER, encER,
                            strIndxPay1, encPay1, strIndxAdm, encAdm, strIndxDisp, encDisp, strIndxMort, encMort, strIndxSurg, encSurg])



In [ ]:
#Fit pipeline model
#2 min run time
model = pipeline.fit(df_manhattan)
df_trnfrm = model.transform(df_manhattan)

In [ ]:
df_trnfrm.show(n=5)

+-------------------+---------------+-----------+--------------------+---------+-------------------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+-----------+--------+-------------+--------+-------------+---------+-------------+-------+-------------+---------+-------------+--------+-------------+---------+--------------+---------+-------------+---------+-------------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|Age Group|Zip Code - 3 digits|Gender| Race|        Ethnicity|Length of Stay|Type of Admission|Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|

In [ ]:
df_trnfrm.groupBy("AgeIndex").count().show(n=100)

+--------+------+
|AgeIndex| count|
+--------+------+
|     0.0|127591|
|     1.0|105148|
|     4.0| 41789|
|     3.0| 69753|
|     2.0| 99621|
+--------+------+



Convert to pandas to reduce categories for columns of interest

In [ ]:

#4 min run time
#@memory_profiler.profile
#def to_pandas():
pandas_df = df_trnfrm.toPandas()
    #print(pandas_df)

#pandas_df = to_pandas()



In [ ]:
pandas_df.head()

,Health Service Area,Hospital County,Facility ID,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,Pay1Index,catVecPay1,AdmIndex,catVecAdm,DispIndex,catVecDisp,MortIndex,catVecMort,SurgIndex,catVecSurg
0,New York City,Manhattan,1437.0,New York Downtown Hospital,0 to 17,100,M,White,Not Span/Hispanic,2,...,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0)",0.0,(1.0)
1,New York City,Manhattan,1437.0,New York Downtown Hospital,0 to 17,100,M,White,Not Span/Hispanic,3,...,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0)",0.0,(1.0)
2,New York City,Manhattan,1437.0,New York Downtown Hospital,0 to 17,100,M,White,Not Span/Hispanic,2,...,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0)",0.0,(1.0)
3,New York City,Manhattan,1437.0,New York Downtown Hospital,0 to 17,100,M,White,Not Span/Hispanic,2,...,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0)",0.0,(1.0)
4,New York City,Manhattan,1437.0,New York Downtown Hospital,0 to 17,100,M,White,Not Span/Hispanic,3,...,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0)",0.0,(1.0)


In [ ]:
##########Patient disposition

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts[class_value_counts<1883].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

Create Dummies

In [ ]:
#TEST THIS IN OPTIMIAZTION STEP I  TIME ALLOWS
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df

Export CSV

In [ ]:
 # The easiest work around of the part file output is to take the data to Pandas and write out a CSV.
# This forces the data to the master node and is not recommended unless you have filtered and/or aggregated your data to a reasonable size.

pandas_df.to_csv('/content/sample_data/inpatient_hospital_2.csv', index=False)


Machine Learning

In [ ]:


X=pandas_df.drop("Total Charges", axis =1)
y= pandas_df["Total Charges"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.